# Hyperparameter Tuning using HyperDrive

TODO: Import Dependencies. In the cell below, import all the dependencies that you will need to complete the project.

In [4]:
# from azureml.core import Workspace, Experiment, ScriptRunConfig
# from azureml.widgets import RunDetails
# from azureml.train.sklearn import SKLearn
# from azureml.train.hyperdrive.run import PrimaryMetricGoal
# from azureml.train.hyperdrive.policy import BanditPolicy
# from azureml.train.hyperdrive.sampling import RandomParameterSampling
# from azureml.train.hyperdrive.runconfig import HyperDriveConfig
# from azureml.train.hyperdrive.parameter_expressions import uniform, choice
# from azureml.train.hyperdrive.parameter_expressions import choice, loguniform, uniform
# from azureml.pipeline.core import Pipeline, PipelineData, TrainingOutput
# from azureml.pipeline.steps import HyperDriveStep, HyperDriveStepRun, PythonScriptStep
# from azureml.core.conda_dependencies import CondaDependencies
# from azureml.core.dataset import Dataset
# from azureml.core.runconfig import RunConfiguration
import os
import shutil
import urllib
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split

# Initialize workspace

In [6]:
# ws = Workspace.from_config()

# # choose a name for experiment
# experiment_name = 'telco-customer-churn-hyperdrive-logistic-regression'
# experiment=Experiment(ws, experiment_name)

# script_folder = './script'
# os.makedirs(script_folder, exist_ok=True)

# print(ws.name, ws.resource_group, ws.location, ws.subscription_id, sep= '\n')

## Dataset

TODO: Get data. In the cell below, write code to access the data you will be using in this project. Remember that the dataset needs to be external.

This dataset contains numerical input variables that come from and PCA algorithm. Due to confidential concerns, the original variables are not available. Features V1, V2, ..., V8 are the outputs of the principal components from the PCA algorithm. The time and amount are the variables that are not transformed by PCA. The target variable is the feature 'Class' and it takes values 1 in case of fraud and 0 otherwise.


This dataset contains transations made by credit cards in September 2012 by European cardholders. This dataset contains 492 frauds out of 284807 transations, and it is highly unbalanced, the positive class (fraud) account for 0.172% of all the transations.


In [7]:


# Try to load the dataset from the Workspace. Otherwise, create it from the file
# NOTE: update the key to match the dataset name
found = False
key = "Customer Churn"
description_text = "Customer Churn DataSet for Udacity Capstone Project"

# if key in ws.datasets.keys(): 
#         found = True
#         dataset = ws.datasets[key] 

# if not found:
#         # Create AML Dataset and register it into Workspace
#         example_data = 'https://raw.githubusercontent.com/srees1988/predict-churn-py/main/customer_churn_data.csv'
#         dataset = Dataset.Tabular.from_delimited_files(example_data)        
#         #Register Dataset in Workspace
#         dataset = dataset.register(workspace=ws,
#                                    name=key,
#                                    description=description_text)


# df = dataset.to_pandas_dataframe()


df = pd.read_csv(
    "https://raw.githubusercontent.com/srees1988/predict-churn-py/main/customer_churn_data.csv"
)

df.describe()

,SeniorCitizen,tenure,MonthlyCharges
count,7043.000000,7043.000000,7043.000000
mean,0.162147,32.371149,64.761692
std,0.368612,24.559481,30.090047
min,0.000000,0.000000,18.250000
25%,0.000000,9.000000,35.500000
50%,0.000000,29.000000,70.350000
75%,0.000000,55.000000,89.850000
max,1.000000,72.000000,118.750000


# Preprocessing data

In [9]:
def clean_data(df):
    df.drop("customerID", axis=1, inplace=True)

    # replace blanks with np.nan
    df["TotalCharges"] = df["TotalCharges"].replace(" ", np.nan)
    # convert to float64
    df["TotalCharges"] = df["TotalCharges"].astype("float64")

    df["TotalCharges"] = df["TotalCharges"].fillna(df["TotalCharges"].median())

    # Replace binary values
    df["gender"] = df["gender"].apply(lambda s: 1 if s == "Female" else 0)
    df["Partner"] = df["Partner"].apply(lambda s: 1 if s == "Yes" else 0)
    df["Dependents"] = df["Dependents"].apply(lambda s: 1 if s == "Yes" else 0)
    df["PhoneService"] = df["PhoneService"].apply(lambda s: 1 if s == "Yes" else 0)
    df["PaperlessBilling"] = df["PaperlessBilling"].apply(
        lambda s: 1 if s == "Yes" else 0
    )
    df["Churn"] = df["Churn"].apply(lambda s: 1 if s == "Yes" else 0)

    MultipleLines = pd.get_dummies(df["MultipleLines"], prefix="MultipleLines")
    df.drop("MultipleLines", inplace=True, axis=1)
    df = df.join(MultipleLines)
    InternetService = pd.get_dummies(df["InternetService"], prefix="InternetService")
    df.drop("InternetService", inplace=True, axis=1)
    df = df.join(InternetService)
    OnlineSecurity = pd.get_dummies(df["OnlineSecurity"], prefix="OnlineSecurity")
    df.drop("OnlineSecurity", inplace=True, axis=1)
    df = df.join(OnlineSecurity)
    OnlineBackup = pd.get_dummies(df["OnlineBackup"], prefix="OnlineBackup")
    df.drop("OnlineBackup", inplace=True, axis=1)
    df = df.join(OnlineBackup)
    DeviceProtection = pd.get_dummies(df["DeviceProtection"], prefix="DeviceProtection")
    df.drop("DeviceProtection", inplace=True, axis=1)
    df = df.join(DeviceProtection)
    TechSupport = pd.get_dummies(df["TechSupport"], prefix="TechSupport")
    df.drop("TechSupport", inplace=True, axis=1)
    df = df.join(TechSupport)
    StreamingTV = pd.get_dummies(df["StreamingTV"], prefix="StreamingTV")
    df.drop("StreamingTV", inplace=True, axis=1)
    df = df.join(StreamingTV)
    StreamingMovies = pd.get_dummies(df["StreamingMovies"], prefix="StreamingMovies")
    df.drop("StreamingMovies", inplace=True, axis=1)
    df = df.join(StreamingMovies)
    Contract = pd.get_dummies(df["Contract"], prefix="Contract")
    df.drop("Contract", inplace=True, axis=1)
    df = df.join(Contract)
    PaymentMethod = pd.get_dummies(df["PaymentMethod"], prefix="PaymentMethod")
    df.drop("PaymentMethod", inplace=True, axis=1)
    df = df.join(PaymentMethod)
    y_df = df.pop("Churn")
    # x_df = df.drop("Churn", inplace=True, axis=1)

    return df, y_df


In [10]:
x, y = clean_data(df)

In [12]:
y.value_counts()

0    5174
1    1869
Name: Churn, dtype: int64

# Train test Split Dataset

In [13]:
# TODO: Split data into train and test sets.
x_train, x_test, y_train, y_test = train_test_split(
    x, y, test_size=0.2, stratify=y, random_state=42
)

train_dataset = pd.concat([x_train,pd.DataFrame(y_train)], axis=1)
test_dataset = pd.concat([x_test,pd.DataFrame(y_test)], axis=1)

# directory = 'train'
# if not os.path.exists(directory):
#     os.makedirs(directory)
    
# directory = 'test'
# if not os.path.exists(directory):
#     os.makedirs(directory)

# # Export data as csv
# train_dataset.to_csv("./train/train_data.csv", index=False)
# test_dataset.to_csv("./test/test_data.csv", index=False)

# # # Upload data to the datastore
# datastore = ws.get_default_datastore()
# datastore.upload(src_dir='./train', target_path = experiment_name)
# datastore.upload(src_dir='./test', target_path = experiment_name)
# print('Data uploaded to DataStore')

# csv_path_train = [(datastore, experiment_name+'/train_data.csv')]
# csv_path_test = [(datastore, experiment_name+'/test_data.csv')]

# train_data = Dataset.Tabular.from_delimited_files(path=csv_path_train)
# test_data = Dataset.Tabular.from_delimited_files(path=csv_path_test)

# display(train_data.to_pandas_dataframe().head())
# display(test_data.to_pandas_dataframe().head())


In [14]:
y_train.value_counts()

0    4139
1    1495
Name: Churn, dtype: int64

In [15]:
y_test.value_counts()

0    1035
1     374
Name: Churn, dtype: int64

## Cluster Provisioning

In [8]:
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.core.compute_target import ComputeTargetException

# TODO: Create compute cluster
# Use vm_size = "Standard_D2_V2" in your provisioning configuration.
# max_nodes should be no greater than 4.


cluster_name = "cluster-vhcg"
# verify that the cluster does not exist already
try:
    cpu_cluster = ComputeTarget(workspace=ws, name = cluster_name)
    print('Found existing cluster, use it.')
except ComputeTargetException:
    compute_config = AmlCompute.provisioning_configuration(vm_size='Standard_D2_V2', max_nodes = 4, idle_seconds_before_scaledown=120)
    cpu_cluster = ComputeTarget.create(ws, cluster_name, compute_config)

cpu_cluster.wait_for_completion(show_output=True)

InProgress....
SucceededProvisioning operation finished, operation "Succeeded"
Succeeded
AmlCompute wait for completion finished

Minimum number of nodes requested have been provisioned


# Copy the training file into the script folder

In [9]:
shutil.copy('./train.py', script_folder)

'./script/train.py'

# Setup an input for the ScriptRunConfig

In [10]:
# data_folder = dataset.as_mount()

# Configure the training job

In [11]:
run_config  = ScriptRunConfig(
    source_directory=script_folder,
    script='train.py',
    compute_target= cpu_cluster
)

# train_step = PythonScriptStep(
#     name='Training_Step',
#     script_name='train.py',
#     compute_target=cpu_cluster,
#     source_directory=script_folder
# )
print("Train step created")

Train step created


## Hyperdrive Configuration

TODO: Explain the model you are using and the reason for chosing the different hyperparameters, termination policy and config settings.

In [17]:
from azureml.widgets import RunDetails
from azureml.train.sklearn import SKLearn
from azureml.train.hyperdrive.run import PrimaryMetricGoal
from azureml.train.hyperdrive.policy import BanditPolicy
from azureml.train.hyperdrive.sampling import RandomParameterSampling
from azureml.train.hyperdrive.runconfig import HyperDriveConfig
from azureml.train.hyperdrive.parameter_expressions import uniform, choice
import os

# Specify parameter sampler
ps = RandomParameterSampling({
    '--solver': choice('newton-cg', 'lbfgs', 'liblinear'),
    #'--penalty': choice('none', 'l1', 'l2'),
    '--C': choice(1e-5, 1e-4, 1e-3, 1e-2, 1e-1, 1, 10, 100),
    '--max_iter': choice(100, 200, 300, 400, 500)
})

# Specify a Policy
policy = BanditPolicy(slack_factor=0.1, evaluation_interval=2, delay_evaluation=10)

if "training" not in os.listdir():
    os.mkdir("./training")

# Create a SKLearn estimator for use with train.py
est = SKLearn(source_directory='./', compute_target=cpu_cluster, entry_script='train.py')

'SKLearn' estimator is deprecated. Please use 'ScriptRunConfig' from 'azureml.core.script_run_config' with your own defined environment or the AzureML-Tutorial curated environment.
'enabled' is deprecated. Please use the azureml.core.runconfig.DockerConfiguration object with the 'use_docker' param instead.


In [18]:
# Create a HyperDriveConfig using the estimator, hyperparameter sampler, and policy.
hyperdrive_config = HyperDriveConfig(
    run_config=run_config,
    hyperparameter_sampling=ps, 
    primary_metric_name='AUC_weighted', 
    primary_metric_goal=PrimaryMetricGoal.MAXIMIZE, 
    max_total_runs=100, 
    max_concurrent_runs=4, 
    max_duration_minutes=10080, 
    policy=policy, 
    
    # estimator=est
)

# HyperDrive Step

In [19]:
metrics_output_name = 'metrics_output'
best_model_output_name = 'best_model_output'
model_output_name = 'model_output'

metrics_data = PipelineData(name='metrics_data',
                           datastore=datastore,
                           pipeline_output_name=metrics_output_name,
                           training_output=TrainingOutput(type='Metrics'))
model_data = PipelineData(name='model_data',
                           datastore=datastore,
                           pipeline_output_name=best_model_output_name,
                           training_output=TrainingOutput(type='Model'))
saved_model = PipelineData(
    name='saved_model',
    pipeline_output_name=model_output_name,
    training_output=TrainingOutput("Model", model_file="outputs/model/saved_mode.pb")
)

In [20]:
hd_step_name='hd_step'
hd_step = HyperDriveStep(
    name = hd_step_name,
    hyperdrive_config=hyperdrive_config,
    estimator_entry_script_arguments=[],
    inputs=[train_data.as_named_input('train_data')], 
    outputs=[metrics_data, saved_model]
)

## Find and register best model

In [21]:
conda_dep = CondaDependencies()
conda_dep.add_pip_package("azureml-sdk")

rcfg = RunConfiguration(conda_dependencies=conda_dep)


register_model_step = PythonScriptStep(
    script_name='register_model.py',
    name='register_model_step',
    inputs=[saved_model],
    compute_target=cpu_cluster,
    arguments=['--saved-model', saved_model],
    allow_reuse=True,
    runconfig=rcfg                                    
)

register_model_step.run_after(hd_step)

## Create and Validate Pipeline

In [22]:
#TODO: Submit your experiment
pipeline = Pipeline(workspace=ws, steps=[hd_step, register_model_step])
pipeline.validate()

Step hd_step is ready to be created [2baf95e9]
Step register_model_step is ready to be created [339a6f20]


[]

# Run the Pipeline

In [23]:

pipeline_run = experiment.submit(pipeline)
# hyperdrive_run = experiment.submit(hyperdrive_config)


Created step hd_step [2baf95e9][65cc31c0-c996-4a4c-8af1-0c0ef9caa6e2], (This step will run and generate new outputs)
Created step register_model_step [339a6f20][7c0b4d32-7b4c-4e7b-b639-ae2ada5fca57], (This step will run and generate new outputs)
Submitted PipelineRun 01fa9a21-1dc4-403a-9504-53f570e75c8c
Link to Azure Machine Learning Portal: https://ml.azure.com/runs/01fa9a21-1dc4-403a-9504-53f570e75c8c?wsid=/subscriptions/a24a24d5-8d87-4c8a-99b6-91ed2d2df51f/resourcegroups/aml-quickstarts-163333/workspaces/quick-starts-ws-163333&tid=660b3398-b80e-49d2-bc5b-ac1dc93b5254


# Monitor using widget

In [24]:
RunDetails(pipeline_run).show()

_PipelineWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO', …

# Wait for the completion of this Pipeline run

In [25]:
pipeline_run.wait_for_completion()

PipelineRunId: 01fa9a21-1dc4-403a-9504-53f570e75c8c
Link to Azure Machine Learning Portal: https://ml.azure.com/runs/01fa9a21-1dc4-403a-9504-53f570e75c8c?wsid=/subscriptions/a24a24d5-8d87-4c8a-99b6-91ed2d2df51f/resourcegroups/aml-quickstarts-163333/workspaces/quick-starts-ws-163333&tid=660b3398-b80e-49d2-bc5b-ac1dc93b5254
PipelineRun Status: NotStarted
PipelineRun Status: Running


StepRunId: 772d76d0-9fc4-4d2c-83d0-6b20c20b7241
Link to Azure Machine Learning Portal: https://ml.azure.com/runs/772d76d0-9fc4-4d2c-83d0-6b20c20b7241?wsid=/subscriptions/a24a24d5-8d87-4c8a-99b6-91ed2d2df51f/resourcegroups/aml-quickstarts-163333/workspaces/quick-starts-ws-163333&tid=660b3398-b80e-49d2-bc5b-ac1dc93b5254
StepRun( hd_step ) Status: Running


## Run Details

OPTIONAL: Write about the different models trained and their performance. Why do you think some models did better than others?

TODO: In the cell below, use the `RunDetails` widget to show the different experiments.

In [6]:
RunDetails(hyperdrive_run).show()
hyperdrive_run.wait_for_completion()

_HyperDriveWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO'…

{'runId': 'HD_57579958-57e8-48a4-a6ed-4e9eeb274d80',
 'target': 'cluster-vhcg',
 'status': 'Completed',
 'startTimeUtc': '2021-10-26T22:52:26.00036Z',
 'endTimeUtc': '2021-10-26T23:31:37.380678Z',
 'services': {},
 'properties': {'primary_metric_config': '{"name": "Accuracy", "goal": "maximize"}',
  'resume_from': 'null',
  'runTemplate': 'HyperDrive',
  'azureml.runsource': 'hyperdrive',
  'platform': 'AML',
  'ContentSnapshotId': '7717604e-23db-4f0f-b2f8-7ab0f46e450b',
  'user_agent': 'python/3.6.9 (Linux-5.4.0-1056-azure-x86_64-with-debian-buster-sid) msrest/0.6.21 Hyperdrive.Service/1.0.0 Hyperdrive.SDK/core.1.34.0',
  'score': '0.8232789212207239',
  'best_child_run_id': 'HD_57579958-57e8-48a4-a6ed-4e9eeb274d80_21',
  'best_metric_status': 'Succeeded'},
 'inputDatasets': [],
 'outputDatasets': [],
 'logFiles': {'azureml-logs/hyperdrive.txt': 'https://mlstrg162142.blob.core.windows.net/azureml/ExperimentRun/dcid.HD_57579958-57e8-48a4-a6ed-4e9eeb274d80/azureml-logs/hyperdrive.txt?sv

In [7]:
assert(hyperdrive_run.get_status() == 'Completed')

## Best Model

TODO: In the cell below, get the best model from the hyperdrive experiments and display all the properties of the model.

In [8]:
import joblib
# Get your best run and save the model from that run.
best_run = hyperdrive_run.get_best_run_by_primary_metric()
best_run_metrics = best_run.get_metrics()
parameter_values = best_run.get_details()['runDefinition']['arguments']

print('Best parameters: ', parameter_values)
print('Best Run Id: ', best_run.id)
print('\n Accuracy:', best_run_metrics['Accuracy'])


Best parameters:  ['--C', '0.1', '--max_iter', '100', '--solver', 'lbfgs']
Best Run Id:  HD_57579958-57e8-48a4-a6ed-4e9eeb274d80_21

 Accuracy: 0.8232789212207239


In [ ]:
#TODO: Save the best model
best_model = best_run.register_model(model_name='udacity_project_model', model_path='./')

## Model Deployment

Remember you have to deploy only one of the two models you trained.. Perform the steps in the rest of this notebook only if you wish to deploy this model.

TODO: In the cell below, register the model, create an inference config and deploy the model as a web service.

TODO: In the cell below, send a request to the web service you deployed to test it.

TODO: In the cell below, print the logs of the web service and delete the service